In [55]:

   
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pickle
import random
from scipy import sparse
import itertools
from scipy.io import savemat, loadmat
import re
import string
from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import pandas as pd
import sys
sys.path.insert(1, '/home/ec2-user/SageMaker/github/aspect_topic_modeling')

from src.models.utils import remove_stopWords

# 20News

In [7]:
# 20News Data
print('reading data...')
train_data = fetch_20newsgroups(subset='train')
test_data = fetch_20newsgroups(subset='test')
len(train_data['data'])

reading data...


11314

In [16]:
def contains_punctuation(w):
    return any(char in string.punctuation for char in w)



init_docs_tr = [re.findall(r'''[\w']+|[.,!?;-~{}`´_<=>:/@*()&'$%#"]''', train_data.data[doc]) for doc in range(len(train_data.data))]
init_docs_ts = [re.findall(r'''[\w']+|[.,!?;-~{}`´_<=>:/@*()&'$%#"]''', test_data.data[doc]) for doc in range(len(test_data.data))]

init_docs = init_docs_tr + init_docs_ts
init_docs = [[w.lower() for w in init_docs[doc] if not contains_punctuation(w)] for doc in range(len(init_docs))]
#init_docs = [[w for w in init_docs[doc] if not contains_numeric(w)] for doc in range(len(init_docs))]
sentences = [[w for w in init_docs[doc] if len(w)>1] for doc in range(len(init_docs))]
#init_docs = [" ".join(init_docs[doc]) for doc in range(len(init_docs))]


In [25]:
#generate word 2 vec models
w2vmodel = Word2Vec(init_docs,vector_size=200, window=10, negative = 5, min_count= 10)

#generate input for the model
vocab = list(set([j for i in init_docs for j in i if j in w2vmodel.wv]))
vocab = [''] + vocab
word_track = {i: ind for ind, i in enumerate(vocab)}
index_track = {ind: i for ind, i in enumerate(vocab)}
#pad the input



In [34]:
np.percentile([len(i) for i in sentences], 85)

381.0

In [37]:

vocab_tensor = torch.Tensor([[0] * 200]  + [w2vmodel.wv[i] for i in vocab[1:]])
vocab_ind = [torch.LongTensor([word_track[it] for it in i if it in word_track][:384]) for i in sentences]
input = torch.nn.utils.rnn.pad_sequence(vocab_ind, batch_first=True, padding_value=0)

#preprocessing to calculate the coherehence
mlb = MultiLabelBinarizer()
XX = mlb.fit_transform([[word_track[it] for it in i if it in word_track]  for i in sentences] + [[0]])


# AG News

In [46]:
D = pd.read_csv('https://raw.githubusercontent.com/yumeng5/WeSTClass/master/agnews/dataset.csv', names = ['class', 'documents', 'text'])

In [51]:
D['clean_text'] = D['documents'] + D['text']

In [56]:
D['clean_text'] = D.swifter.apply(lambda x: ' '.join(remove_stopWords(x['documents'] + x['text'])).split(), axis=1)


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

In [57]:
#generate word 2 vec models
w2vmodel = Word2Vec(D['clean_text'].values,vector_size=200, window=10, negative = 5, min_count= 10)

#generate input for the model
vocab = list(set([j for i in init_docs for j in i if j in w2vmodel.wv]))
vocab = [''] + vocab
word_track = {i: ind for ind, i in enumerate(vocab)}
index_track = {ind: i for ind, i in enumerate(vocab)}
#pad the input


# RCV

In [58]:
from sklearn.datasets import fetch_rcv1
rcv1 = fetch_rcv1()

In [ ]:
#cannot use

In [ ]:
#backup dataset NYT and Arxiv

#https://github.com/yumeng5/JoSH/tree/master/datasets
